# Bybit 인증

In [3]:
# your api_key & api_secret
from bybit import api_key, api_secret
from pybit.usdt_perpetual import HTTP

session = HTTP(
    endpoint='https://api.bybit.com',
    api_key=api_key,
    api_secret=api_secret
)
import time
import pandas as pd
import torch
from pprint import pprint

# 취급하는 코인 정보 확인하기

In [ ]:
query_symbol = session.query_symbol()
pprint(query_symbol)

# 심볼의 최신정보 확인

In [ ]:
pprint(session.latest_information_for_symbol(symbol="BTCUSDT"))

# 내 잔고

In [ ]:
rst = session.get_wallet_balance(coin="USDT")
pprint(rst)

# 현재까지의 PnL 확인하기

In [ ]:
ncpal = session.closed_profit_and_loss(
    symbol="BTCUSDT",
)
pprint(ncpal)

# 주문하기

### Open Long

In [ ]:
resp = session.place_active_order(
    symbol="BTCUSDT",
    side="Buy", # 'Buy' or 'Sell'
    order_type="Market", # 'Limit' or 'Market'
    qty=0.001,
    time_in_force="GoodTillCancel",  # 'GoodTillCancel' or 'ImmediateOrCancel' or 'FillOrKill' or 'PostOnly'
    reduce_only=False,
    close_on_trigger=False,
    take_profit=20000,
)

print(resp)

### Close Long

In [ ]:
resp = session.place_active_order(
    symbol="BTCUSDT",
    side="Sell", # 'Buy' or 'Sell'
    order_type="Market", # 'Limit' or 'Market'
    qty=0.001,
    time_in_force="GoodTillCancel",  # 'GoodTillCancel' or 'ImmediateOrCancel' or 'FillOrKill' or 'PostOnly'
    reduce_only=True,
    close_on_trigger=True,
)

print(resp)

### Open Short

In [ ]:
resp = session.place_active_order(
    symbol="BTCUSDT",
    side="Sell", # 'Buy' or 'Sell'
    order_type="Market", # 'Limit' or 'Market'
    qty=100,
    time_in_force="GoodTillCancel",  # 'GoodTillCancel' or 'ImmediateOrCancel' or 'FillOrKill' or 'PostOnly'
    reduce_only=False,
    close_on_trigger=False,
    take_profit=10000,
    stop_loss=30000,
)

print(resp)

### Close Short

In [ ]:
resp = session.place_active_order(
    symbol="BTCUSDT",
    side="Buy", # 'Buy' or 'Sell'
    order_type="Market", # 'Limit' or 'Market'
    qty=0.001,
    time_in_force="GoodTillCancel",  # 'GoodTillCancel' or 'ImmediateOrCancel' or 'FillOrKill' or 'PostOnly'
    reduce_only=True,
    close_on_trigger=True,
)

print(resp)

# 내 포지션 확인

In [ ]:
my_position = session.my_position(
    symbol="BTCUSDT"
)
pprint(my_position)

# 포지션 모드 변경

In [ ]:
position_mode_switch = session.position_mode_switch(
    symbol="BTCUSDT",
    mode="BothSide" # 'BothSide' or 'MergedSingle'
)

pprint(position_mode_switch)

# 레버리지 세팅

In [18]:
set_leverage = session.set_leverage(
    symbol="BTCUSDT",
    buy_leverage=1,
    sell_leverage=1
)

# 리스크 한계 구하기

In [ ]:
get_risk_limit = session.get_risk_limit(
    symbol="BTCUSDT"
)
pprint(get_risk_limit)

# 1분봉 모으기

In [ ]:
result = {}
while True:
    nowtime = int(time.time())
    i = 0
    while True:
        i += 1
        if i % 100 == 0:
            print((nowtime//60 - 200*i + 1) * 60)
        resp = session.query_kline(
            symbol="BTCUSDT",
            interval="1",
            limit=200,
            from_time=(nowtime//60 - 200*i + 1) * 60
        )
        df = resp['result']
        df = pd.DataFrame(df)
        dfl = df[['open', 'high', 'low', 'close', 'volume']].to_numpy().tolist()
        df = {k: v for k,v in zip(df['id'], dfl)}
        before_len = len(result)
        result.update(df)
        if before_len == len(result):
            break
    if int(time.time()) - nowtime < 60:
        break

rst = sorted(result.items(), key=lambda x:x[0])
rst = [i[1] for i in rst]
print(len(rst))

torch.save(result, "bybit_data.pt")